In [57]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

fatal: destination path 'Qualcomm-DL-Hackathon' already exists and is not an empty directory.


In [58]:
import pandas as pd

train_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
train_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [59]:
test_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
test_data.head()
#train_data.shape

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [60]:
sample_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/sample_submission.csv')
sample_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,0
2,2082.jpg,0
3,808.jpg,0
4,1907.jpg,0


In [61]:
import zipfile
images_dir = '/content/Qualcomm-DL-Hackathon/train/images'
with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-1.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-2.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import torch.nn.functional as F

# Custom dataset class
class EmergencyVehicleDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, is_train=True): # Add is_train flag
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.is_train = is_train # Store the flag

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name)

        if self.is_train: # Check if it's training data
            label = int(self.annotations.iloc[idx, 1])
        else:
            label = 0 # Dummy label for test data


        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the dataset
train_csv_dir = '/content/Qualcomm-DL-Hackathon/train/train.csv'
test_csv_dir = '/content/Qualcomm-DL-Hackathon/test.csv'
train_dataset = EmergencyVehicleDataset(csv_file=train_csv_dir, root_dir='/content/Qualcomm-DL-Hackathon/dataset/images', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = EmergencyVehicleDataset(csv_file=test_csv_dir, root_dir='/content/Qualcomm-DL-Hackathon/dataset/images', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 30 * 30, 128)
        self.fc2 = nn.Linear(128, 1)  # Output layer for binary classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 30 * 30)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

model = CNN()

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(10):  # Number of epochs
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)

        # Reshape labels to have the same shape as outputs
        labels = labels.unsqueeze(1).float()  # Add a dimension and convert to float

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')


Epoch 1, Loss: 0.6622909175661894
Epoch 2, Loss: 0.484517844250569
Epoch 3, Loss: 0.42323186764350307
Epoch 4, Loss: 0.34515899104567677
Epoch 5, Loss: 0.2693215465316406
Epoch 6, Loss: 0.17201633364535296
Epoch 7, Loss: 0.11365780391945289
Epoch 8, Loss: 0.07315429027837056
Epoch 9, Loss: 0.03944469669547219
Epoch 10, Loss: 0.03044606595694159
Finished Training


In [72]:
class EmergencyVehicleDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, is_train=True): # Add is_train flag
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.is_train = is_train # Store the flag

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name)

        # Check if 'emergency_or_not' column exists before accessing it
        if self.is_train and 'emergency_or_not' in self.annotations.columns:
            label = int(self.annotations.iloc[idx, 1])
        else:
            label = 0 # Dummy label for test data


        if self.transform:
            image = self.transform(image)

        return image, label

In [73]:
test_dataset = EmergencyVehicleDataset(csv_file='/content/Qualcomm-DL-Hackathon/test.csv',
                            root_dir='/content/Qualcomm-DL-Hackathon/dataset/images',
                            transform=transform)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(test_dataset)

# Make predictions on the test data
predictions = []
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Create a submission dataframe
submission_df = pd.DataFrame({'image_names': test_data['image_names'], 'emergency_or_not': predictions})
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
